In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_log_error, mean_squared_error
import joblib
from pathlib import Path
import os



## Model Evaluation 

In [14]:

class ModelEvaluator:
    def __init__(self, project_root=None):
        """Initialize evaluator with project paths
        Args:
            project_root: Optional path to project root. If None, will try to detect automatically.
        """
        # Setup paths
        if project_root is None:
            try:
                # Try script-style path resolution
                self.project_root = Path(__file__).parents[3]
            except NameError:
                # Fallback for notebook environment
                current_dir = Path(os.getcwd())
                # Assuming notebook is in src/predictions/prices/
                self.project_root = current_dir.parents[3]
        else:
            self.project_root = Path(project_root)
            
        self.models_dir = self.project_root / "models/saved"
        self.test_data_dir = self.project_root / "models/test_data"
        
        # Verify paths exist
        if not self.models_dir.exists() or not self.test_data_dir.exists():
            raise FileNotFoundError(
                f"Required directories not found. Please ensure the following paths exist:\n"
                f"Models dir: {self.models_dir}\n"
                f"Test data dir: {self.test_data_dir}"
            )
        
        # Load model and scaler
        try:
            self.model = load_model(self.models_dir / "best_model.keras")
            self.scaler = joblib.load(self.models_dir / "scaler.save")
        except Exception as e:
            raise Exception(f"Error loading model or scaler: {str(e)}")
        
        # Load test data
        try:
            self.X_test = np.load(self.test_data_dir / "X_test.npy")
            self.y_test = np.load(self.test_data_dir / "y_test.npy")
            self.test_timestamps = np.load(
                self.test_data_dir / "test_timestamps.npy", 
                allow_pickle=True
            )
        except Exception as e:
            raise Exception(f"Error loading test data: {str(e)}")

    def evaluate(self):
        # -------------------------------
        # 3. Make Predictions on the Test Set
        # -------------------------------
        y_pred = self.model.predict(self.X_test)

        # Inverse transform predictions and true values to original scale
        y_pred_inv = self.scaler.inverse_transform(y_pred)
        y_test_inv  = self.scaler.inverse_transform(self.y_test)

        # -------------------------------
        # 4. Calculate Error Metrics
        # -------------------------------
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
        # msle = mean_squared_log_error(y_test_inv, y_pred_inv)
        mse  = mean_squared_error(y_test_inv, y_pred_inv)
        rmse = mse**0.5

        print(f"MAPE: {mape:.4f}")
        # print(f"MSLE: {msle:.4f}")
        print(f"MSE: {mse:.4f}")
        print(f"RMSE: {rmse:.4f}")

        # Calculate a custom peak error (mean error on the top 10% values)
        def peak_error(y_true, y_pred):
            y_true_series = pd.Series(y_true.flatten())
            threshold = y_true_series.quantile(0.9)
            peak_indices = y_true_series[y_true_series > threshold].index
            return np.mean(np.abs(y_true.flatten()[peak_indices] - y_pred.flatten()[peak_indices]))

        peak_err = peak_error(y_test_inv, y_pred_inv)
        print(f"Peak Error: {peak_err:.4f}")

        # -------------------------------
        # 5. Plot Predictions vs. Actual Values
        # -------------------------------
        plt.figure(figsize=(15, 5))
        plt.plot(self.test_timestamps, y_test_inv, label="Actual Prices")
        plt.plot(self.test_timestamps, y_pred_inv, label="Predicted Prices", linestyle="dashed")
        plt.title("LSTM Forecasting: Actual vs. Predicted Prices (Test Data)")
        plt.xlabel("Time")
        plt.ylabel("Price")
        plt.legend()
        plt.show()

## Run Evaluation

In [15]:

project_root = Path(os.getcwd()).parents[2]  # Adjust the number of parents as needed
evaluator = ModelEvaluator(project_root=project_root)

evaluator.evaluate()



1013/1013 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step


ValueError: Expected 2D array, got 1D array instead:
array=[0.07146429 0.07144103 0.07122011 ... 0.09823142 0.08383624 0.07208056].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.